In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
!pip install usearch
!pip install torch datasets bitsandbytes
!pip install git+https://github.com/huggingface/transformers
!pip install accelerate
!pip install sentence-transformers
!pip install -i https://test.pypi.org/simple/ bitsandbytes

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0crh90us
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0crh90us
  Resolved https://github.com/huggingface/transformers to commit bc72b4e2cdcbc80d5f56731f35dbc9c18b4c8de6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://test.pypi.org/simple/


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/truy van thong tin dpt/ImageRetrieval

/content/drive/MyDrive/Colab Notebooks/truy van thong tin dpt/ImageRetrieval


In [ ]:
import torch
from datasets import load_dataset
from transformers import InstructBlipForConditionalGeneration, InstructBlipProcessor

In [ ]:
model = InstructBlipForConditionalGeneration.from_pretrained(
    'Salesforce/instructblip-vicuna-7b',
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
)
processor = InstructBlipProcessor.from_pretrained(
    'Salesforce/instructblip-vicuna-7b',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
import os

In [ ]:
from PIL import Image
import requests

In [ ]:
# datasets =[('detection-datasets/fashionpedia',None,'val')]
datasets = [('phiyodr/coco2017', None, 'validation')]
prompt1 = "describe this image in full detail, describe each and every aspect of the image so the artist could re create the image"
prompt2 = "create an extensive description of this image"
counter = 169
for name, config, split in datasets:
    d = load_dataset(name, config, split=split)
    for idx in range(500, len(d)):
        url = d[idx]["coco_url"]
        image = Image.open(requests.get(url, stream=True).raw)
        # image = d[idx]["image"]
        desc = ''
        for _prompt in [prompt1, prompt2]:
            inputs = processor(
                images=image,
                text=_prompt,
                return_tensors='pt'
            ).to(model.device, torch.bfloat16)
            outputs = model.generate(
                **inputs,
                do_sample=False,
                num_beams=10,
                max_length=512,
                min_length=16,
                top_p=0.9,
                repetition_penalty=1.5,
                temperature=1,
            )
            generated_text = processor.batch_decode(
                outputs,
                skip_special_tokens=True,
            )[0].strip()

            desc += generated_text + " "

        desc = desc.strip()
        if not os.path.exists('images'):
          os.makedirs('images')
        image.save(f'images/{counter}.jpg')
        print(counter, desc)

        with open("description.csv", 'a') as f:
            f.write(f"{counter}, {desc}\n")

        counter += 1
        if counter >= 500:
            break

        torch.cuda.empty_cache()


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


64 based on the image description The image depicts a woman wearing a white tennis outfit playing tennis on a grassy surface. She is actively engaged in the game, running and swinging her tennis racket to hit the ball. There are several tennis balls scattered around the scene, indicating that the game is in progress. In total, there are nine tennis balls visible in the image, with some located closer to the woman and others positioned further away. a woman swinging a tennis racket at a ball on a tennis court
65 based on the image description a young girl is seen brushing her teeth with a blue toothbrush. She is wearing a green shirt and appears to be focused on brushing her teeth diligently. In addition to the toothbrush, there are two cups visible in the scene, one on the left side of the image and another on the right side. The presence of these cups suggests that the girl might be using them to rinse her mouth after brushing her teeth. Overall, the image depicts a young girl taking 

In [ ]:
from sentence_transformers import SentenceTransformer
import pickle
import pandas as pd

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

with open('description.csv', 'r') as f:
    lines = f.readlines()

lines = [line.strip().split(',') for line in lines]

for idx, line in enumerate(lines):
    lines[idx] = [line[0], ','.join(line[1:])]

df = pd.DataFrame(lines, columns=['id', 'desc'])
embeddings = model.encode(df['desc'].tolist(), show_progress_bar=True)
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)




# Search


In [ ]:
from usearch.index import Index

# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)
embeddings = embeddings.astype('float32')

index = Index(ndim=embeddings.shape[1], metric="cos", dtype="f16")
num_results = 5

for i, vector in enumerate(embeddings):
    index.add(i, vector)

index.save("index.usearch")

def _search(query):
    query_embedding = model.encode(query)
    query_embedding = np.array(query_embedding).astype('float32')
    query_embedding = query_embedding.reshape(1, -1)
    matches = index.search(query_embedding, num_results)
    images = [f'images/{i}.jpg' for i in matches.keys]
    return images

with gr.Blocks() as demo:
    query = gr.Textbox(lines=1, label='search query')
    outputs = gr.Gallery(preview=True)
    submit = gr.Button(value='search')
    submit.click(_search, inputs=query, outputs=outputs)

demo.launch()